# 拡散モデル: GMM

## シード

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
from torch.optim import Adam


np.random.seed(42) # 乱数生成用のシードを設定
ite = 1 # 乱数生成の回数

random_seed = np.random.randint(0, 10000, ite)  # ランダムな整数値をシード値として取得.例えば 0 〜 9999 の間の整数をite個生成
print("random_seed", random_seed) # 乱数シード値の確認


## モデル構築

### 正弦波位置エンコーディング

In [ ]:
# --- 時間埋め込み（正弦波位置エンコーディング）---
# output_dim が偶数であることを前提とするか、奇数にも対応できるよう調整
def pos_encoding(timesteps, output_dim, device='cpu'):
    # timestepsをfloat型かつ2次元テンソル (batch_size, 1) に変換
    # dtypeを明示的に指定
    position = timesteps.unsqueeze(1).float().to(device)

    # div_term の計算 (output_dim が奇数の場合を考慮し、output_dim // 2 とする)
    # output_dim が奇数の場合、最後の次元は0でパディングするか、または無視される
    div_term = torch.exp(torch.arange(0, output_dim, 2, device=device, dtype=torch.float32) * (-np.log(10000.0) / output_dim))

    # sin と cos の計算
    sin_vals = torch.sin(position * div_term)
    cos_vals = torch.cos(position * div_term)

    # 結合し、output_dim に合うように調整
    sinusoid = torch.cat([sin_vals, cos_vals], dim=1)

    # もし output_dim が奇数で、かつその次元が重要なら調整が必要
    # 例えば、output_dim が17なら、sinusoid は16次元になる。
    # 完全に output_dim と同じ次元にするならパディングするか、output_dim は偶数に固定すべき
    if sinusoid.shape[1] < output_dim:
        # 残りの次元を0でパディング (例: output_dim=17 の場合、1次元足りない)
        padding = torch.zeros(sinusoid.shape[0], output_dim - sinusoid.shape[1], device=device, dtype=torch.float32)
        sinusoid = torch.cat([sinusoid, padding], dim=1)
    elif sinusoid.shape[1] > output_dim:
        # 偶数で初期化したが、output_dimが小さい場合 (例: output_dim=16だが、計算上18になってしまった場合)
        # これは通常発生しないが、安全のため
        sinusoid = sinusoid[:, :output_dim]

    return sinusoid

### 拡散モデル

In [ ]:
# --- 拡散モデル ---
class DiffusionModel(nn.Module):
    # input_data_dim を引数として追加
    def __init__(self, input_data_dim, time_embed_dim=16, hidden_dim=64, dropout_rate=0.1, activation='leaky_relu'):
        super(DiffusionModel, self).__init__()
        self.time_embed_dim = time_embed_dim
        self.input_data_dim = input_data_dim # 入力データの次元を保持

        # 活性化関数の選択
        if activation == 'leaky_relu':
            self.activation_fn = nn.LeakyReLU()
        elif activation == 'elu':
            self.activation_fn = nn.ELU()
        else: # default to ReLU
            self.activation_fn = nn.ReLU()

        # 各層にBatch NormalizationとDropoutを追加
        # fc1の入力次元を input_data_dim + time_embed_dim に変更
        self.fc1 = nn.Linear(self.input_data_dim + time_embed_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim) # BatchNorm1d は特徴量次元に適用
        self.dropout1 = nn.Dropout(dropout_rate)

        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.dropout2 = nn.Dropout(dropout_rate)

        # 最終出力層の次元を input_data_dim に変更
        self.fc3 = nn.Linear(hidden_dim, self.input_data_dim)

    def forward(self, x, t):
        # time_embed_dim は self.time_embed_dim を使用
        t_embed = pos_encoding(t, self.time_embed_dim, x.device)

        x_t = torch.cat([x, t_embed], dim=-1)

        # Batch Normalization -> Activation -> Dropout の順
        x_t = self.fc1(x_t)
        x_t = self.bn1(x_t)
        x_t = self.activation_fn(x_t)
        x_t = self.dropout1(x_t)

        x_t = self.fc2(x_t)
        x_t = self.bn2(x_t)
        x_t = self.activation_fn(x_t)
        x_t = self.dropout2(x_t)

        return self.fc3(x_t)

# --- 拡散プロセス ---
class Diffuser:
    def __init__(self, num_timesteps=1000, beta_start=0.0001, beta_end=0.02, device='cpu'):
        self.num_timesteps = num_timesteps
        self.device = device
        self.betas = torch.linspace(beta_start, beta_end, num_timesteps, device=device)
        self.alphas = 1 - self.betas
        self.alpha_bars = torch.cumprod(self.alphas, dim=0)

    def add_noise(self, x_0, t):
        # t_idx は (batch_size,) のテンソルを想定
        # alpha_bars から適切な alpha_bar を取得するために gather を使用
        t_idx = t - 1 # alphas[0] is for t=1
        alpha_bar = self.alpha_bars.gather(0, t_idx).view(-1, 1) # (N, 1)

        noise = torch.randn_like(x_0, device=self.device)
        x_t = torch.sqrt(alpha_bar) * x_0 + torch.sqrt(1 - alpha_bar) * noise
        return x_t, noise

    def denoise(self, model, x, t):
        T = self.num_timesteps
        # tがtensorの場合のassert
        assert torch.all(t >= 1) and torch.all(t <= T)

        t_idx = t - 1 # alphas[0] is for t=1

        # alpha と alpha_bar も gather を使用
        alpha = self.alphas.gather(0, t_idx).view(-1, 1)
        alpha_bar = self.alpha_bars.gather(0, t_idx).view(-1, 1)

        model.eval()
        with torch.no_grad():
            eps = model(x, t)

        # DDPM論文のサンプリングステップの計算
        # x_{t-1} の平均項
        mu = (x - (1 - alpha) / torch.sqrt(1 - alpha_bar) * eps) / torch.sqrt(alpha)

        # x_{t-1} の分散項 (標準偏差)
        # DDPMでは、最終ステップ(t=1)以外はノイズを追加
        # beta_t_tilde = (1 - alpha_bar_{t-1}) / (1 - alpha_bar_t) * beta_t
        # 一般的にはベータの分散を用いることが多い
        # 例えば、beta_t = self.betas[t_idx]
        sigma = torch.sqrt(self.betas.gather(0, t_idx)).view(-1, 1) # sqrt(beta_t)

        # t=1 の場合はノイズを加えない (x_0を生成する最終ステップ)
        # tがtensorの場合の条件分岐
        z = torch.randn_like(x, device=self.device)
        z[t == 1] = 0 # t=1のバッチ要素のみzを0にする

        x_prev = mu + sigma * z

        return x_prev # x_{t-1} を返す

## GMM学習データ生成

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def generate_uncorrelated_gmm_2d_toy_dataset(num_total_samples=2000,
                                            ratio_mode1=0.5, # モード1のデータが全体の何割を占めるか (0.0 ~ 1.0)
                                            mu1=np.array([-3.0, -3.0]),
                                            sigma1_diag=np.array([1.0, 1.0]), # 対角成分のみ指定
                                            mu2=np.array([3.0, 3.0]),
                                            sigma2_diag=np.array([1.0, 1.0]), # 対角成分のみ指定
                                            random_seed=42):
    """
    2つの無相関な2次元正規分布を組み合わせたガウス混合モデル (GMM) からトイデータセットを生成します。
    各正規分布の比率を指定できます。

    Args:
        num_total_samples (int): 生成する総サンプル数。
        ratio_mode1 (float): モード1 (1つ目の正規分布) のデータが全体の何割を占めるか (0.0 ~ 1.0)。
                              モード2の比率は (1 - ratio_mode1) になります。
        mu1 (np.ndarray): 1つ目の正規分布の平均ベクトル (2次元)。
        sigma1_diag (np.ndarray): 1つ目の正規分布の共分散行列の対角成分 (2要素)。
                                  非対角成分は0と仮定されます。
        mu2 (np.ndarray): 2つ目の正規分布の平均ベクトル (2次元)。
        sigma2_diag (np.ndarray): 2つ目の正規分布の共分散行列の対角成分 (2要素)。
                                  非対角成分は0と仮定されます。
        random_seed (int): 乱数生成のシード。

    Returns:
        np.ndarray: 生成されたデータポイント (N x 2)。
    """
    if not (0.0 <= ratio_mode1 <= 1.0):
        raise ValueError("ratio_mode1 must be between 0.0 and 1.0")

    np.random.seed(random_seed)

    # 各モードのサンプル数を計算
    num_samples_mode1 = int(num_total_samples * ratio_mode1)
    num_samples_mode2 = num_total_samples - num_samples_mode1 # 残りはモード2

    # 共分散行列を作成 (対角成分のみ有効)
    sigma1 = np.diag(sigma1_diag)
    sigma2 = np.diag(sigma2_diag)

    print(f"Generating {num_total_samples} samples:")
    print(f"  Mode 1 ({ratio_mode1*100:.1f}%): {num_samples_mode1} samples")
    print(f"  Mode 2 ({(1-ratio_mode1)*100:.1f}%): {num_samples_mode2} samples")

    # 1つ目の正規分布からデータを生成
    data_mode1 = np.random.multivariate_normal(mu1, sigma1, num_samples_mode1)

    # 2つ目の正規分布からデータを生成
    data_mode2 = np.random.multivariate_normal(mu2, sigma2, num_samples_mode2)

    # 両方のデータを結合
    dataset = np.vstack((data_mode1, data_mode2))

    # データをシャッフル (モードの偏りをなくすため)
    np.random.shuffle(dataset)

    return dataset

def plot_2d_dataset(dataset, title="2D GMM Toy Dataset (Uncorrelated)"):
    """
    2次元データセットをプロットします。

    Args:
        dataset (np.ndarray): プロットするデータセット (N x 2)。
        title (str): グラフのタイトル。
    """
    plt.figure(figsize=(8, 6))
    plt.scatter(dataset[:, 0], dataset[:, 1], s=10, alpha=0.7)
    plt.title(title)
    plt.xlabel("Dimension 1")
    plt.ylabel("Dimension 2")
    plt.grid(True)
    plt.axis('equal') # x軸とy軸のスケールを合わせる
    plt.show()

### 生成実行

In [ ]:
# 例4: 比率30:70、円形クラスタ
print("--- Example 4: 30:70 Ratio, Circular Clusters ---")
dataset4 = generate_uncorrelated_gmm_2d_toy_dataset(
    num_total_samples=2000,
    ratio_mode1=0.3,
    mu1=np.array([-3.0, -3.0]),
    sigma1_diag=np.array([1.0, 1.0]),
    mu2=np.array([3.0, 3.0]),
    sigma2_diag=np.array([1.0, 1.0])
)
plot_2d_dataset(dataset4, title="Uncorrelated GMM (30:70, Circular)")

In [ ]:
# 7:3混合正規分布の密度関数
from matplotlib.pylab import multivariate_normal


def gmm_density(x, mu1, sigma1_diag, mu2, sigma2_diag, ratio_mode1):
    """
    2つの無相関な正規分布の混合密度関数を計算します。

    Args:
        x (np.ndarray): 入力データポイント (N x 2)。
        mu1 (np.ndarray): 1つ目の正規分布の平均ベクトル (2次元)。
        sigma1_diag (np.ndarray): 1つ目の正規分布の共分散行列の対角成分 (2要素)。
        mu2 (np.ndarray): 2つ目の正規分布の平均ベクトル (2次元)。
        sigma2_diag (np.ndarray): 2つ目の正規分布の共分散行列の対角成分 (2要素)。
        ratio_mode1 (float): モード1の比率。
    
    Returns:
        np.ndarray: 各データポイントに対する混合密度関数の値。
    """
    # 共分散行列を作成
    sigma1 = np.diag(sigma1_diag)
    sigma2 = np.diag(sigma2_diag)

    # 各モードの密度を計算
    density_mode1 = multivariate_normal.pdf(x, mean=mu1, cov=sigma1)
    density_mode2 = multivariate_normal.pdf(x, mean=mu2, cov=sigma2)

    # 混合密度を計算
    mixed_density = ratio_mode1 * density_mode1 + (1 - ratio_mode1) * density_mode2

    return mixed_density

In [ ]:
mu1 = np.array([3.0, 3.0])
mu2 = np.array([-3.0, -3.0])
top_density = gmm_density(mu1, mu2)

## カーネル密度推定(KDE)

In [ ]:
# カーネル密度推定のための関数
def plot_kde_2d(dataset, title="2D KDE Plot"):
    """
    2次元データセットのカーネル密度推定 (KDE) プロットを作成します。

    Args:
        dataset (np.ndarray): プロットするデータセット (N x 2)。
        title (str): グラフのタイトル。
    """
    from scipy.stats import gaussian_kde

    # KDEを計算
    kde = gaussian_kde(dataset.T)
    x_min, x_max = dataset[:, 0].min(), dataset[:, 0].max()
    y_min, y_max = dataset[:, 1].min(), dataset[:, 1].max()
    
    x_grid = np.linspace(x_min, x_max, 100)
    y_grid = np.linspace(y_min, y_max, 100)
    X, Y = np.meshgrid(x_grid, y_grid)
    Z = kde(np.vstack([X.ravel(), Y.ravel()])).reshape(X.shape)

    # プロット
    plt.figure(figsize=(8, 6))
    plt.contourf(X, Y, Z, levels=20, cmap='viridis')
    plt.colorbar(label='Density')
    plt.scatter(dataset[:, 0], dataset[:, 1], s=10, alpha=0.5, color='white')
    plt.title(title)
    plt.xlabel("Dimension 1")
    plt.ylabel("Dimension 2")
    plt.grid(True)
    plt.axis('equal') # x軸とy軸のスケールを合わせる
    plt.show()
# KDEプロットの例
print("--- KDE Plot for Example 4 ---")
plot_kde_2d(dataset4, title="KDE Plot for Uncorrelated GMM (30:70, Circular)")

## 学習

In [ ]:
# ハイパーパラメータ
seed = 42 # 乱数シード値
num_timesteps = 1000 # 拡散ステップ数
epochs = 20          # 学習エポック数
lr = 5e-3         # 学習率
device = 'cuda' if torch.cuda.is_available() else 'cpu'
time_embed_dim = 16
input_data_dim = dataset4.shape[1] # データセットの次元数
hidden_dim = 64 # 隠れ層の次元数
model = DiffusionModel(input_data_dim=input_data_dim, time_embed_dim=time_embed_dim, hidden_dim=hidden_dim).to(device)
optimizer = Adam(model.parameters(), lr=lr)
diffuser = Diffuser(num_timesteps=num_timesteps, device=device)

# 学習データ(ガウスノイズ)
print("############################################")
start_time = time.time() # 計測開始
# print(f"Data_Set_{i+1}, Seed: {seed}") # 開始の合図

print(f"拡散ステップ数: {num_timesteps}, 学習エポック数: {epochs}, 学習率: {lr}")
print("############################################")
np.random.seed(seed) # 取得した乱数を新しいシード値として設定
data = dataset4 # 例4のデータセットを使用
print("data", len(data))
batch_size = 64 # バッチサイズ
# モデル学習に使う DataLoader も float32 のテンソルから作成
dataloader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)


# データの要約
print("dataの平均ベクトル", np.mean(data, axis=0)) # 平均ベクトル
print("dataの分散共分散行列", np.cov(data.T)) # 分散共分散行列
print("dataの相関係数", np.corrcoef(data.T)) # 相関係数

# 学習
losses = []
for epoch in range(epochs):
    loss_sum = 0.0
    for batch in dataloader:
        optimizer.zero_grad()
        x = batch.to(device).float()
        t = torch.randint(1, num_timesteps + 1, (len(x),), device=device)

        x_noisy, noise = diffuser.add_noise(x, t)
        x_noisy = x_noisy.float()
        noise = noise.float()
        noise_pred = model(x_noisy, t)
        loss = F.mse_loss(noise_pred, noise)

        loss.backward()
        optimizer.step()

        loss_sum += loss.item()
    avg_loss = loss_sum / len(dataloader)
    losses.append(avg_loss)
    # 5の倍数エポックで損失を表示
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}, Loss: {avg_loss}")
print("学習終了")
end_time = time.time() # 計測終了
print('\n')
print(f"学習時間: {end_time - start_time:.2f}秒")


# 学習曲線のプロット
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Trained By data_by_seed_{}'.format(seed))
plt.show()
print('\n')
print("#"*50)
print('\n')

## サンプリングの実行

In [ ]:
import numpy as np
import torch
from tqdm import tqdm

# (前回のDiffuser, DiffusionModel, pos_encodingの定義がここにあると仮定)
# Diffuserインスタンスを初期化 (num_timesteps を明示的に渡す)
# モデルの初期化もここで行われると仮定
# 例:
# num_timesteps = 1000 # 定義が必要
# diffuser = Diffuser(num_timesteps=num_timesteps, device=device)
# model = DiffusionModel(time_embed_dim=16, hidden_dim=64, dropout_rate=0.1, activation='leaky_relu').to(device)
# model.load_state_dict(torch.load('your_model_weights.pth')) # 学習済みモデルをロードする場合

# 拡散モデルのサンプリング関数
def generate_samples(model, diffuser_instance, n_samples=1000, n_batches=100, device='cpu'):
    """
    学習済み拡散モデルを用いてサンプルを生成します。

    Args:
        model (nn.Module): 学習済みの拡散モデル（ノイズ予測器）。
        diffuser_instance (Diffuser): 拡散プロセスを管理するDiffuserインスタンス。
        n_samples (int): 各生成バッチで生成するサンプル数。
        n_batches (int): 何回生成プロセスを繰り返すか（生成するデータセットの数）。
        device (str): テンソルを配置するデバイス ('cuda' または 'cpu')。

    Returns:
        np.ndarray: 生成されたサンプル。形状は (n_batches, n_samples, 2)。
    """
    model.eval() # モデルを評価モードに設定
    
    new_sample_list = [] # 生成された各バッチのサンプルを格納するリスト
    
    # tqdm の記述をより明確に
    print(f"Generating {n_batches} batches, each with {n_samples} samples...")
    for _ in tqdm(range(n_batches), desc="Generating Samples"):
        # 各生成バッチで異なる初期ノイズから開始
        # torch.manual_seed はここでは不要（異なるランダムノイズが欲しい場合）
        samples = torch.randn((n_samples, 2), device=device) # x_T (完全なノイズ)

        # 拡散ステップを逆に進める (T -> T-1 -> ... -> 1)
        for t in range(diffuser_instance.num_timesteps, 0, -1):
            # 現在の時刻情報をテンソルに変換
            # t_tensorは、バッチ内の各サンプルに対して同じ時刻 't' を持つ
            t_tensor = torch.full((n_samples,), t, device=device, dtype=torch.long)
            
            # ノイズ除去ステップ (x_t から x_{t-1} を計算)
            # diffuser.denoise はモデルの評価モードとno_gradを自身で管理しないように変更済みと仮定
            # generate_samples 関数が全体のno_gradを管理する
            with torch.no_grad(): # 全体の生成プロセスを no_grad ブロックで囲む
                samples = diffuser_instance.denoise(model, samples, t_tensor)
            
        # 最終的にデノイズされたサンプル (x_0) をリストに追加
        new_sample_list.append(samples.cpu().numpy())
        
    return np.array(new_sample_list) # (n_batches, n_samples, 2) のNumPy配列


if __name__ == "__main__":
    # --- デバイスの設定 ---
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    # # --- 拡散モデルとDiffuserのパラメータ設定 ---
    # num_timesteps = 1000 
    # beta_start = 0.0001
    # beta_end = 0.02
    # time_embed_dim = 16
    # hidden_dim = 64
    # dropout_rate = 0.1
    # activation = 'leaky_relu'

    # # --- Diffuserとモデルのインスタンス化 ---
    # diffuser = Diffuser(num_timesteps=num_timesteps, beta_start=beta_start, beta_end=beta_end, device=device)
    # model = DiffusionModel(time_embed_dim=time_embed_dim, hidden_dim=hidden_dim, 
    #                        dropout_rate=dropout_rate, activation=activation).to(device)

    # --- (注意) ここに学習済みモデルのロード処理が入ります ---
    # 例: model.load_state_dict(torch.load("path_to_your_trained_model.pth"))
    # ロードしない場合、モデルは初期状態なのでランダムなサンプルしか生成しません。
    print("WARNING: Model is not loaded with trained weights. Generating random samples.")

    # --- サンプリングの実行 ---
    generated_data = generate_samples(
        model=model,
        diffuser_instance=diffuser,
        n_samples=2000,    # 1バッチあたりのサンプル数
        n_batches=50,      # 生成するバッチ数
        device=device
    )

    print(f"Shape of generated_data: {generated_data.shape}") # (n_batches, n_samples, 2)

In [ ]:
print("generated_data.shape", generated_data.shape)
print("generated_data[0].shape", generated_data[0].shape) # (1000, 2)
print("generated_data.shape[0]", generated_data.shape[0]) # バッチ数
print("generated_data[0, :, 0].shape", generated_data[0, :, 0].shape) # 2次元のサンプルデータ

In [ ]:
if __name__ == "__main__":
  # 生成されたデータの一部を可視化（最初のバッチ）
  import matplotlib.pyplot as plt

  if generated_data.shape[0] > 0:
      plt.figure(figsize=(8, 6))
      plt.scatter(generated_data[0, :, 0], generated_data[0, :, 1], s=10, alpha=0.7)
      plt.title("Example of Generated Samples (First Batch)")
      plt.xlabel("Dimension 1")
      plt.ylabel("Dimension 2")
      plt.grid(True)
      plt.axis('equal')
      plt.show()

      # 複数のバッチを可視化して多様性を確認
      if generated_data.shape[0] > 1:
          fig, axes = plt.subplots(1, 2, figsize=(16, 6))
          axes[0].scatter(generated_data[0, :, 0], generated_data[0, :, 1], s=10, alpha=0.7)
          axes[0].set_title("Generated Batch 1")
          axes[0].set_xlabel("Dimension 1")
          axes[0].set_ylabel("Dimension 2")
          axes[0].set_aspect('equal', adjustable='box')
          axes[0].grid(True)

          axes[1].scatter(generated_data[1, :, 0], generated_data[1, :, 1], s=10, alpha=0.7)
          axes[1].set_title("Generated Batch 2")
          axes[1].set_xlabel("Dimension 1")
          axes[1].set_ylabel("Dimension 2")
          axes[1].set_aspect('equal', adjustable='box')
          axes[1].grid(True)
          plt.tight_layout()
          plt.show()

## 生成したデータをKDEにより表示

In [ ]:
print("generated_data[0].shape", generated_data[0].shape) # (1000, 2)

In [ ]:
# KDEプロットの例
print("--- KDE Plot for Generated Samples ---")
plot_kde_2d(generated_data[0], title="KDE Plot for Generated Samples (First Batch)")

### 最初のバッチでKDE→頂点の距離を実行

In [ ]:
from scipy.stats import gaussian_kde
kde_generated_data = gaussian_kde(generated_data[0].T)

# --- 評価グリッドの定義 ---
# generated_data[0] の範囲に基づいてグリッドを生成
x_min, x_max = generated_data[0][:, 0].min() - 1, generated_data[0][:, 0].max() + 1
y_min, y_max = generated_data[0][:, 1].min() - 1, generated_data[0][:, 1].max() + 1

# グリッドの解像度
num_points = 100
X, Y = np.meshgrid(np.linspace(x_min, x_max, num_points),
                   np.linspace(y_min, y_max, num_points))

# KDEをグリッド上で評価
# (2, num_points*num_points) の形状に変換してkdeに渡す
Z = kde_generated_data(np.vstack([X.ravel(), Y.ravel()]))
Z = Z.reshape(X.shape) # 元のグリッド形状に戻す

# --- 2つの山の頂点を特定 (前回のコードから再利用) ---
# 1. 最初のピーク（グローバル最大値）を見つける
max_z_idx_flat = np.argmax(Z)
row1, col1 = np.unravel_index(max_z_idx_flat, Z.shape)
peak1_coords_2d = np.array([X[row1, col1], Y[row1, col1]])
peak1_density = Z[row1, col1]

# 2. 最初のピーク周辺をマスクして2番目のピークを見つける
Z_masked = Z.copy()
mask_radius = 4 # マスクする半径 (GMMの分散やモード間距離に応じて調整)

distances_from_peak1 = np.sqrt((X - peak1_coords_2d[0])**2 + (Y - peak1_coords_2d[1])**2)
Z_masked[distances_from_peak1 < mask_radius] = -1e10 # 負の大きな値に設定して最大値検索から除外

max_z_masked_idx_flat = np.argmax(Z_masked)
row2, col2 = np.unravel_index(max_z_masked_idx_flat, Z_masked.shape)
peak2_coords_2d = np.array([X[row2, col2], Y[row2, col2]])
peak2_density = Z[row2, col2] # マスク前のZから実際の密度を取得

# --- 3次元空間における距離を計算 ---
# ピークの3D座標を定義 (X, Y, Density)
peak1_coords_3d = np.array([peak1_coords_2d[0], peak1_coords_2d[1], peak1_density])
peak2_coords_3d = np.array([peak2_coords_2d[0], peak2_coords_2d[1], peak2_density])

# 3Dユークリッド距離を計算
distance_3d = np.linalg.norm(peak1_coords_3d - peak2_coords_3d)

print(f"最初のピークの座標: ({peak1_coords_3d[0]:.2f}, {peak1_coords_3d[1]:.2f}, 密度(z): {peak1_coords_3d[2]:.4f})")
print(f"2番目のピークの座標: ({peak2_coords_3d[0]:.2f}, {peak2_coords_3d[1]:.2f}, 密度(z): {peak2_coords_3d[2]:.4f})")
print(f"2D空間におけるピーク頂点間の距離: {np.linalg.norm(peak1_coords_2d - peak2_coords_2d):.2f}")
print(f"3D空間におけるピーク頂点間の距離 (密度をz軸として): {distance_3d:.4f}")

# --- 3Dプロット ---
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# サーフェスプロット
ax.plot_surface(X, Y, Z, cmap='viridis', edgecolor='none', alpha=0.8)

ax.scatter(peak1_coords_3d[0], peak1_coords_3d[1], peak1_coords_3d[2], color='red', s=100, label='Peak 1', zorder=10)
ax.scatter(peak2_coords_3d[0], peak2_coords_3d[1], peak2_coords_3d[2], color='blue', s=100, label='Peak 2', zorder=10)

# ラベルとタイトル
ax.set_xlabel('Dimension 1')
ax.set_ylabel('Dimension 2')
ax.set_zlabel('Density')
ax.set_title('3D Kernel Density Estimate from Generated Data')

# 視点の調整 (任意)
ax.view_init(elev=30, azim=150) # 仰角と方位角

# 保存
# plt.savefig('kde_3d_generated_data_plot.png')
# print("KDE_generated_dataに対する3Dの図示を 'kde_3d_generated_data_plot.png' として保存しました。")


### それぞれのバッチで距離を算出

In [ ]:
# 距離をまとめるリスト
distances = []

for i in range(generated_data.shape[0]):
    print(f"generated_data[{i}].shape", generated_data[i].shape) # (1000, 2)
    from scipy.stats import gaussian_kde
    target_data = generated_data[i] # 各バッチのデータを取得
    kde_generated_data = gaussian_kde(target_data.T)

    # --- 評価グリッドの定義 ---
    # generated_data[i] の範囲に基づいてグリッドを生成
    x_min, x_max = target_data[:, 0].min() - 1, target_data[:, 0].max() + 1
    y_min, y_max = target_data[:, 1].min() - 1, target_data[:, 1].max() + 1

    # グリッドの解像度
    num_points = 100
    X, Y = np.meshgrid(np.linspace(x_min, x_max, num_points),
                      np.linspace(y_min, y_max, num_points))

    # KDEをグリッド上で評価
    # (2, num_points*num_points) の形状に変換してkdeに渡す
    Z = kde_generated_data(np.vstack([X.ravel(), Y.ravel()]))
    Z = Z.reshape(X.shape) # 元のグリッド形状に戻す

    # --- 2つの山の頂点を特定 (前回のコードから再利用) ---
    # 1. 最初のピーク（グローバル最大値）を見つける
    max_z_idx_flat = np.argmax(Z)
    row1, col1 = np.unravel_index(max_z_idx_flat, Z.shape)
    peak1_coords_2d = np.array([X[row1, col1], Y[row1, col1]])
    peak1_density = Z[row1, col1]

    # 2. 最初のピーク周辺をマスクして2番目のピークを見つける
    Z_masked = Z.copy()
    mask_radius = 4 # マスクする半径 (GMMの分散やモード間距離に応じて調整)

    distances_from_peak1 = np.sqrt((X - peak1_coords_2d[0])**2 + (Y - peak1_coords_2d[1])**2)
    Z_masked[distances_from_peak1 < mask_radius] = -1e10 # 負の大きな値に設定して最大値検索から除外

    max_z_masked_idx_flat = np.argmax(Z_masked)
    row2, col2 = np.unravel_index(max_z_masked_idx_flat, Z_masked.shape)
    peak2_coords_2d = np.array([X[row2, col2], Y[row2, col2]])
    peak2_density = Z[row2, col2] # マスク前のZから実際の密度を取得

    # --- 3次元空間における距離を計算 ---
    # ピークの3D座標を定義 (X, Y, Density)
    peak1_coords_3d = np.array([peak1_coords_2d[0], peak1_coords_2d[1], peak1_density])
    peak2_coords_3d = np.array([peak2_coords_2d[0], peak2_coords_2d[1], peak2_density])

    # 3Dユークリッド距離を計算
    distance_3d = np.linalg.norm(peak1_coords_3d - peak2_coords_3d)

    print(f"最初のピークの座標: ({peak1_coords_3d[0]:.2f}, {peak1_coords_3d[1]:.2f}, 密度(z): {peak1_coords_3d[2]:.4f})")
    print(f"2番目のピークの座標: ({peak2_coords_3d[0]:.2f}, {peak2_coords_3d[1]:.2f}, 密度(z): {peak2_coords_3d[2]:.4f})")
    print(f"2D空間におけるピーク頂点間の距離: {np.linalg.norm(peak1_coords_2d - peak2_coords_2d):.2f}")
    print(f"3D空間におけるピーク頂点間の距離 (密度をz軸として): {distance_3d:.4f}")

    # 距離をリストに追加
    distances.append(distance_3d)

    # # --- 3Dプロット ---
    # fig = plt.figure(figsize=(10, 8))
    # ax = fig.add_subplot(111, projection='3d')

    # # サーフェスプロット
    # ax.plot_surface(X, Y, Z, cmap='viridis', edgecolor='none', alpha=0.8)

    # ax.scatter(peak1_coords_3d[0], peak1_coords_3d[1], peak1_coords_3d[2], color='red', s=100, label='Peak 1', zorder=10)
    # ax.scatter(peak2_coords_3d[0], peak2_coords_3d[1], peak2_coords_3d[2], color='blue', s=100, label='Peak 2', zorder=10)

    # # ラベルとタイトル
    # ax.set_xlabel('Dimension 1')
    # ax.set_ylabel('Dimension 2')
    # ax.set_zlabel('Density')
    # ax.set_title('3D Kernel Density Estimate from Generated Data')

    # # 視点の調整 (任意)
    # ax.view_init(elev=30, azim=150) # 仰角と方位角

    # 保存
    # plt.savefig('kde_3d_generated_data_plot.png')
    # print("KDE_generated_dataに対する3Dの図示を 'kde_3d_generated_data_plot.png' として保存しました。")

print("距離算出完了")

In [ ]:
# distancesの平均と標準偏差を計算
mean_distance = np.mean(distances)
std_distance = np.std(distances)
print(f"ブートストラップサンプル間の3D空間におけるピーク頂点間の距離の平均: {mean_distance:.4f}")
print(f"ブートストラップサンプル間の3D空間におけるピーク頂点間の距離の標準偏差: {std_distance:.4f}")

In [ ]:
# 算出した距離の分布
plt.figure(figsize=(8, 6))
plt.hist(distances, bins=30, color='skyblue', edgecolor='black', alpha=0.7)
plt.title('Distribution of Distances Between Peaks')
plt.xlabel('Distance')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()